# Playground

## Import

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

In [3]:
import tvh_check_upcoming_recs as check

## Load DataFrame

In [4]:
df = check.get_df()
df

ATTRIBUTES,Title,Channel,Start,Stop
INDEX,,,,
0,"Love, Fashion, Repeat",City,2023-07-25 03:00:00,2023-07-25 05:00:00
1,Les looney tunes,TVA,2023-07-25 05:30:00,2023-07-25 06:00:00
2,Les looney tunes,TVA,2023-07-25 11:28:00,2023-07-25 11:58:00
3,Entre deux draps,Noovo,2023-07-25 19:30:00,2023-07-25 20:00:00
4,Les looney tunes,TVA,2023-07-26 05:30:00,2023-07-26 06:00:00
5,SkyMed,CBC,2023-07-26 21:00:00,2023-07-26 22:00:00
6,Les looney tunes,TVA,2023-07-27 05:30:00,2023-07-27 06:00:00
7,L'échappée belle,TVA,2023-07-27 09:29:00,2023-07-27 11:43:00
8,L'échappée,TVA,2023-07-27 13:00:00,2023-07-27 14:00:00


## Analysis

In [5]:
#                   INDEX : [ 0, 1, 2, 3, 4, 5, 6]
df = pd.DataFrame({'Start': [ 0, 1, 2, 3, 4, 5, 6],
                   'Stop' : [ 1, 6, 3, 5, 5, 6, 7]})
df.sort_values(by='Start', inplace=True, ignore_index=True)
df.rename_axis('ATTRIBUTES', axis='columns', inplace=True)
df.rename_axis('INDEX', axis='index', inplace=True)
df

ATTRIBUTES,Start,Stop
INDEX,,
0,0,1
1,1,6
2,2,3
3,3,5
4,4,5
5,5,6
6,6,7


In [6]:
##### TEMP for development
#df = df.head(5)
#df = df.iloc[1:4]
#df.reset_index(drop=True, inplace=True)
#df

In [7]:
start_c = df['Start'].to_numpy()
stop_c = df['Stop'].to_numpy()
start_r = df['Start'].to_numpy()[:, np.newaxis]
stop_r = df['Stop'].to_numpy()[:, np.newaxis]

In [8]:
# df Analysis

# ref rec:          ************                   : comp overlap ref?
# comp rec:   -------                              : n
#              ----------                          : y
#              -----------------------             : y
#                   ------------                   : y
#                      ------                      : y
#                         ------------             : y
#                              ----------------    : n
#

dfa = pd.DataFrame(((start_c < stop_r) & (stop_c > start_r)) * 1)
dfa.rename_axis('INDEX', axis='columns', inplace=True)
dfa.rename_axis('INDEX', axis='index', inplace=True)
dfa

INDEX,0,1,2,3,4,5,6
INDEX,,,,,,,
0,1,0,0,0,0,0,0
1,0,1,1,1,1,1,0
2,0,1,1,0,0,0,0
3,0,1,0,1,1,0,0
4,0,1,0,1,1,0,0
5,0,1,0,0,0,1,0
6,0,0,0,0,0,0,1


In [9]:
dfa.sum()

INDEX
0    1
1    5
2    2
3    3
4    3
5    2
6    1
dtype: int64

In [10]:
dfr = df.copy()
dfr['NbTunerRequired'] = dfa.sum()
dfr

ATTRIBUTES,Start,Stop,NbTunerRequired
INDEX,,,
0,0,1,1
1,1,6,5
2,2,3,2
3,3,5,3
4,4,5,3
5,5,6,2
6,6,7,1


In [11]:
dfr[dfr['NbTunerRequired'] > 2]

ATTRIBUTES,Start,Stop,NbTunerRequired
INDEX,,,
1,1,6,5
3,3,5,3
4,4,5,3


In [12]:
df

ATTRIBUTES,Start,Stop
INDEX,,
0,0,1
1,1,6
2,2,3
3,3,5
4,4,5
5,5,6
6,6,7


In [13]:
check.analyze_df(df)

ATTRIBUTES,Start,Stop,NbTunerRequired
INDEX,,,
1,1,6,5
3,3,5,3
4,4,5,3


## All-in-one

In [14]:
check.analyze_df(check.get_df())

ATTRIBUTES,Title,Channel,Start,Stop,NbTunerRequired
INDEX,,,,,
